# Project: The Impact of Vaccines on Covid-19 Death Rates

## Project Introduction:

## Changes Made:

## Data Cleaning:

## Exploratory Data Analysis:

## Visualization:

## Machine Learning Analysis:

## Reflection:

Import the Necessary Libraries for Data Analysis and Visualization

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
%matplotlib inline
import matplotlib.pyplot as plt

Step 1: Load the Datasets to use in the project.

In [99]:
main = pd.read_csv("WHO-COVID-19-global-data.csv")
vaccine = pd.read_csv("vaccination-data.csv")
vaccine.head(3)

# vaccine.loc[:, "DATA_SOURCE"].value_counts()

,COUNTRY,ISO3,WHO_REGION,DATA_SOURCE,DATE_UPDATED,TOTAL_VACCINATIONS,PERSONS_VACCINATED_1PLUS_DOSE,TOTAL_VACCINATIONS_PER100,PERSONS_VACCINATED_1PLUS_DOSE_PER100,PERSONS_FULLY_VACCINATED,PERSONS_FULLY_VACCINATED_PER100,VACCINES_USED,FIRST_VACCINE_DATE,NUMBER_VACCINES_TYPES_USED
0,Afghanistan,AFG,EMRO,REPORTING,2022-02-27,5535254,4907058.0,14.219,12.605,4231984.0,10.871,"Beijing CNBG - BBIBP-CorV,Janssen - Ad26.COV 2...",2021-02-22,4.0
1,Albania,ALB,EURO,REPORTING,2022-02-20,2707658,1284034.0,94.100,45.118,1192155.0,41.889,"AstraZeneca - Vaxzevria,Gamaleya - Gam-Covid-V...",2021-01-13,5.0
2,Algeria,DZA,AFRO,REPORTING,2022-02-20,13631683,7456361.0,31.086,17.004,6076272.0,13.857,"Beijing CNBG - BBIBP-CorV,Gamaleya - Gam-Covid...",2021-01-30,4.0


Assumption 1: We changed the WHO_region and Country in **main** to match the WHO_REGION and COUNTRY in **vaccine**. <br>
Assumption 2: We will analyze/visualize data for 237 countries featured in the main dataset.

In [101]:
main = main.rename(columns={"WHO_region": "WHO_REGION", "Country": "COUNTRY"})
main = main.merge(vaccine, how='outer', on=['WHO_REGION', 'COUNTRY'])
main = main.dropna(subset=['Date_reported'])
# main.loc[:, "COUNTRY"].value_counts()
# main

In [23]:
main.loc[:, "Country"].value_counts()          # There are a total of 789 dates for 237 countries

Afghanistan        789
Paraguay           789
Nigeria            789
Niue               789
North Macedonia    789
                  ... 
Grenada            789
Guadeloupe         789
Guam               789
Guatemala          789
Zimbabwe           789
Name: Country, Length: 237, dtype: int64

## To Do List: 
-Create a subset of adjusted data <br>
-Remove NaN Values <br>
-Check if Unreported dates for vaccination are (ir)relevant <br>
-Remove Country Code, ISO3
-Maybe Remove Vaccine types and vaccine types used

Assumption 3: Remove Columns That We Don't Need

In [94]:
main = main.drop(columns=['Country_code', 'ISO3', 'DATA_SOURCE'])   # Cleaned Data

In [102]:
# main.isnull().sum().sum()        # Check how many null values we have; We have a lot!
# main